EDA

We will start with the ETL stage. Unlike the EDA, where only an exploration of the data was performed, here in the ETL, is where the relevant changes are made, so that at the end you have a reliable database, columns that are not appropriate for the type of query to be performed in the api will be eliminated. This in order to consolidate data that is easy to understand. 

We loaded all the necessary notebooks for the ETL process.

In [3]:
import pandas as pd
import numpy as np


The file data_string.csv is loaded into a dataframe named data.

In [4]:
df_amazon = pd.read_csv('Datasets/amazon_prime_titles.csv')
df_hulu = pd.read_csv('Datasets/hulu_titles.csv')
df_disney = pd.read_csv('Datasets/disney_plus_titles.csv')
df_netflix = pd.read_json('./Datasets/netflix_titles.json')

A column is created to identify the streaming service.

In [5]:
df_amazon = df_amazon.assign(Platform ="amazon")
df_disney = df_disney.assign(Platform ="disney")
df_hulu = df_hulu.assign(Platform ="hulu")
df_netflix = df_netflix.assign(Platform ="netflix")

All dataframes are merged into one, for better management.

In [6]:
data_string = pd.concat([df_amazon, df_disney, df_hulu, df_netflix], axis=0)

Rename columns. 

In [7]:
cambios={'type': 'Category', 
         'title': 'Title',
         'director': 'Director',
         'cast': 'Cast',
         'country': 'Country',
         'date_added': 'Date_added',
         'release_year': 'Release_year',
         'rating': 'Rating',
         'duration': 'Duration',
         'listed_in': 'Genre',
         'description': 'Synopsis'
         }


data_string.rename(columns = cambios, inplace=True)

Type (change name to "category" so it doesn't clash with mysql).

Date format of the Date_added column.

In [8]:
data_string["Date_added"]=data_string["Date_added"].str.replace(",", "").str.strip() ## quitar espacios delante y detrás
data_string["Date_added"]=pd.to_datetime(data_string["Date_added"], format="%B %d %Y")

It was found that some data belonging to the "duration" column are located in the "ranting" column.
We proceeded to change their location so as not to lose them.

In [9]:
data_string['Duration'] = data_string.apply(lambda row: row['Rating'] if ((pd.isnull(row['Duration'])) & (("min" in str(row['Rating'])) or ("Season" in str(row['Rating'])))) else row['Duration'], axis=1)




Another alternative...

In [10]:
#data_string.loc[data_string.Rating.str.contains('min'),'Duration (Min)'] = data_string.loc[data_string.Rating.str.contains('min'),'Rating']
#data_string.loc[data_string.Rating.str.contains('min'), 'Rating'] = "No_information"

In [11]:
data_string.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22998 entries, 0 to 8806
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   show_id       22998 non-null  object        
 1   Category      22998 non-null  object        
 2   Title         22998 non-null  object        
 3   Director      14739 non-null  object        
 4   Cast          17677 non-null  object        
 5   Country       11499 non-null  object        
 6   Date_added    13444 non-null  datetime64[ns]
 7   Release_year  22998 non-null  int64         
 8   Rating        22134 non-null  object        
 9   Duration      22811 non-null  object        
 10  Genre         22998 non-null  object        
 11  Synopsis      22994 non-null  object        
 12  Platform      22998 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(11)
memory usage: 2.5+ MB


We control the percentage of nulls in our columns.

In [12]:
round(data_string.isna().mean()*100,2)

show_id          0.00
Category         0.00
Title            0.00
Director        35.91
Cast            23.14
Country         50.00
Date_added      41.54
Release_year     0.00
Rating           3.76
Duration         0.81
Genre            0.00
Synopsis         0.02
Platform         0.00
dtype: float64

In [13]:
data_string.head()

,show_id,Category,Title,Director,Cast,Country,Date_added,Release_year,Rating,Duration,Genre,Synopsis,Platform
0,s1,Movie,The Grand Seduction,Don McKellar,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",Canada,2021-03-30,2014,NaN,113 min,"Comedy, Drama",A small fishing village must procure a local d...,amazon
1,s2,Movie,Take Care Good Night,Girish Joshi,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",India,2021-03-30,2018,13+,110 min,"Drama, International",A Metro Family decides to fight a Cyber Crimin...,amazon
2,s3,Movie,Secrets of Deception,Josh Webber,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...",United States,2021-03-30,2017,NaN,74 min,"Action, Drama, Suspense",After a man discovers his wife is cheating on ...,amazon
3,s4,Movie,Pink: Staying True,Sonia Anderson,"Interviews with: Pink, Adele, Beyoncé, Britney...",United States,2021-03-30,2014,NaN,69 min,Documentary,"Pink breaks the mold once again, bringing her ...",amazon
4,s5,Movie,Monster Maker,Giles Foster,"Harry Dean Stanton, Kieran O'Brien, George Cos...",United Kingdom,2021-03-30,1989,NaN,45 min,"Drama, Fantasy",Teenage Matt Banting wants to work with a famo...,amazon



As you can see the "duration" nulls decreased from 482 to 187, which is a 61.2% decrease in the number of nulls.

Next we will analyze query by query to see which columns are necessary to build our database. 

* Maximum duration by type of film (movie/series), by platform and by year: The request should be: get_max_duration(year, platform, [min or season]).

    * Title.
    * Category.
    * Platform.    
    * Release_year.
    * Duration (separate the series data with the min of the movies).


* Number of movies and series (separated) per platform The request should be: get_count_plataform(platform).

    * Category.
    * Platform. 


* Number of times a genre and platform is repeated with the highest frequency of the genre. The request should be: get_listedin('genre').

    * Title.
    * Genre.
    * Platform.  



As an example of genre you can use 'comedy', which should return a cunt of 2099 for the amazon platform.


* Repeating actor according to platform and year. The request should be: get_actor(platform, year).

    * Cast
    * Platform 
    * Release_year  


The columns that we see are not useful in the queries to be made, or that have much missing data, will be eliminated.  

In [14]:
data_string = data_string.drop(["show_id","Director","Country","Date_added","Rating"],axis=1)

In [15]:
data_string.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22998 entries, 0 to 8806
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Category      22998 non-null  object
 1   Title         22998 non-null  object
 2   Cast          17677 non-null  object
 3   Release_year  22998 non-null  int64 
 4   Duration      22811 non-null  object
 5   Genre         22998 non-null  object
 6   Synopsis      22994 non-null  object
 7   Platform      22998 non-null  object
dtypes: int64(1), object(7)
memory usage: 1.6+ MB


The original indexes are deleted.

In [16]:
data_string = data_string.reset_index(drop=True)

There are two main categories in this Datasets, movies and TV series. The movies have as time minutes and the series have seasons. This must be separated depending on the category. 

For this, each word will be separated from the Duration column, where each one will be stored word by word in a column of a new dataframe. 

In [17]:
separate = data_string.Duration.str.split(" ", n=1, expand=True)


The information of the duration column has been extracted and separated. We will proceed to create two new columns in data_string. We will proceed to enter the information. 

In [18]:
data_string["Duration"] = separate[0]
data_string["Type_duration"] = separate[1]

In [19]:
data_string.isnull().sum()

Category            0
Title               0
Cast             5321
Release_year        0
Duration          187
Genre               0
Synopsis            4
Platform            0
Type_duration     187
dtype: int64

At this point all the columns are needed, but we can see that there are still null values, so they will be replaced by no data.

In [20]:
data_string = data_string.fillna("undetermined")

Check for duplicate rows. Not found. 

In [21]:
duplicate = data_string[data_string.duplicated()==True].count()
print(duplicate)

Category         0
Title            0
Cast             0
Release_year     0
Duration         0
Genre            0
Synopsis         0
Platform         0
Type_duration    0
dtype: int64


With the following statement we pass it to float and all the "no data" will be passed to NaN.

In [22]:
data_string["Duration"] = pd.to_numeric(data_string["Duration"], errors="coerce")

We replace the nulls by 0. This in order to be effective when performing our searches.

In [23]:
data_string["Duration"] = data_string["Duration"].fillna(0)

We change the format from float to int.

In [24]:
data_string["Duration"]=data_string["Duration"].astype(int)

A new column is created in the dataframe and named "Id_Stream". This will contain a unique identifier for each movie. 

In [25]:
data_string["Id_Stream"] = data_string.index

We arrange the columns in a more convenient way.

In [26]:
data_string = data_string[['Id_Stream','Title','Category','Duration','Type_duration',
                           'Release_year','Genre', 'Cast', 'Synopsis', 'Platform']
                            ]

We remove the blanks in the following columns.

In [27]:
data_string["Category"] = data_string["Category"].str.strip()
data_string["Title"] = data_string["Title"].str.strip()
data_string["Type_duration"] = data_string["Type_duration"].str.strip()

In [28]:
data_string

,Id_Stream,Title,Category,Duration,Type_duration,Release_year,Genre,Cast,Synopsis,Platform
0,0,The Grand Seduction,Movie,113,min,2014,"Comedy, Drama","Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",A small fishing village must procure a local d...,amazon
1,1,Take Care Good Night,Movie,110,min,2018,"Drama, International","Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",A Metro Family decides to fight a Cyber Crimin...,amazon
2,2,Secrets of Deception,Movie,74,min,2017,"Action, Drama, Suspense","Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...",After a man discovers his wife is cheating on ...,amazon
3,3,Pink: Staying True,Movie,69,min,2014,Documentary,"Interviews with: Pink, Adele, Beyoncé, Britney...","Pink breaks the mold once again, bringing her ...",amazon
4,4,Monster Maker,Movie,45,min,1989,"Drama, Fantasy","Harry Dean Stanton, Kieran O'Brien, George Cos...",Teenage Matt Banting wants to work with a famo...,amazon
...,...,...,...,...,...,...,...,...,...,...
22993,22993,Zodiac,Movie,158,min,2007,"Cult Movies, Dramas, Thrillers","Mark Ruffalo, Jake Gyllenhaal, Robert Downey J...","A political cartoonist, a crime reporter and a...",netflix
22994,22994,Zombie Dumb,TV Show,2,Seasons,2018,"Kids' TV, Korean TV Shows, TV Comedies",undetermined,"While living alone in a spooky town, a young g...",netflix
22995,22995,Zombieland,Movie,88,min,2009,"Comedies, Horror Movies","Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",Looking to survive in a world taken over by zo...,netflix
22996,22996,Zoom,Movie,88,min,2006,"Children & Family Movies, Comedies","Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...","Dragged from civilian life, a former superhero...",netflix


UPLOAD TO DATABASE

The next step is to extract the result of our ETL process, MYSQL will be used to store all the data.

In [29]:
data_string.to_csv("Datasets/data_string.csv")